In [1]:
import sys
import math
import os
import shutil

In [2]:
import import_ipynb
from Generator_MatrixVectorMultiply_M_N_PR_noHR_mamcs_baseAdderSize_NVL_VL_v2_5 import MatrixVectorMultiply_noHR_MAMCS

importing Jupyter notebook from Generator_MatrixVectorMultiply_M_N_PR_noHR_mamcs_baseAdderSize_NVL_VL_v2_5.ipynb
importing Jupyter notebook from DotProduct_noHR_C_V3_7.ipynb
importing Jupyter notebook from DotProduct_Systolic_V1_11.ipynb
importing Jupyter notebook from AdderTree_ASm_V1_20.ipynb
importing Jupyter notebook from RegisteredNto1Adder_V1_1.ipynb
importing Jupyter notebook from DotProduct_noHR_M_V3_7.ipynb


In [3]:
M = input('Enter M: ')
N = input('Enter N: ')
PR = input('Enter No Of Parallel Row (empty for 1): ')
MAMCS = input('Enter Multiply-Add Max Component Size (empty for {}): '.format(N))
AdderSize = input('Enter maximum size allowed for base adders (empty for 2): ')
Enable_Vector_Latch = input('Enable Vector Latch? (empty or 0 to disable, 1 to enable): ')
IN_WIDTH = input('Enter IN_WIDTH default value (empty for 10): ')
INPUT_REG_DEPTH = input('Enter INPUT_REG_DEPTH default value (empty for 1): ')
MULT_PIPE_DEPTH = input('Enter MULT_PIPE_DEPTH default value (empty for 1): ')
M=int(M)
N=int(N)
PR=int(PR)
#MAMCS=int(MAMCS)
#AdderSize=int(AdderSize)
#Enable_Vector_Latch=int(Enable_Vector_Latch)
if not IN_WIDTH:
    IN_WIDTH=10
else:
    IN_WIDTH = int(IN_WIDTH)
if not PR:
    PR=1
else:
    PR = int(PR)
#INPUT_REG_DEPTH=int(INPUT_REG_DEPTH)
#MULT_PIPE_DEPTH=int(MULT_PIPE_DEPTH)
lgN=math.ceil(math.log(N,2))
OUT_WIDTH=2*IN_WIDTH+lgN
parameters={
    "N":N,
    "M":M,
    "PR":PR,
    "MAMCS":MAMCS,
    "AdderSize":AdderSize,
    "Enable_Vector_Latch":Enable_Vector_Latch,
    "INPUT_REG_DEPTH":INPUT_REG_DEPTH,
    "MULT_PIPE_DEPTH":MULT_PIPE_DEPTH,
    #"ENABLE_INPUT_REGISTERS":EnableInputRegisters,
    "IN_WIDTH":IN_WIDTH
}
IN_DATA_LENGHT=(PR+1)*N*IN_WIDTH  #PR shows how many rows is taken simultaneously
OUT_DATA_LENGHT=PR*(OUT_WIDTH)
Print_To_File=1
Files_Location='MatrixVectorMultiplyGeneratedVerilogCode/'
Is_Top=1
modules={}
GeneratedFileName=MatrixVectorMultiply_noHR_MAMCS(parameters,Print_To_File,Files_Location,Is_Top,modules)
Files_Location+=GeneratedFileName+"/"
wrapperModuleName=GeneratedFileName+"_AXIStream"
of=open(Files_Location+wrapperModuleName+".v", 'w+')

Enter M: 10
Enter N: 13
Enter No Of Parallel Row (empty for 1): 10
Enter Multiply-Add Max Component Size (empty for 13): 13
Enter maximum size allowed for base adders (empty for 2): 
Enable Vector Latch? (empty or 0 to disable, 1 to enable): 
Enter IN_WIDTH default value (empty for 10): 
Enter INPUT_REG_DEPTH default value (empty for 1): 
Enter MULT_PIPE_DEPTH default value (empty for 1): 


In [4]:
# in this cell we will use the generated file and instanciate it in our new AXI wrapper module.
print("`timescale 1ns / 1ps\n",file=of)
print("module "+wrapperModuleName,file=of)
print("#(",file=of)
print("parameter IN_DATA_LENGHT= {}, ".format(IN_DATA_LENGHT),file=of)
print("parameter OUT_DATA_LENGHT= {} ".format(OUT_DATA_LENGHT),file=of)
print(")( \n",file=of)
print("input aclk,".format(),file=of)
print("input aresetn,".format(),file=of)
print("input enable,".format(),file=of)
print("input [IN_DATA_LENGHT-1:0]s_axi_data,".format(),file=of)
print("input s_axi_valid,".format(),file=of)
print("input m_axi_ready,".format(),file=of)
print("output reg[OUT_DATA_LENGHT-1:0]m_axi_data,".format(),file=of)
print("output reg m_axi_valid,".format(),file=of)
print("output reg s_axi_ready".format(),file=of)
print(");\n".format(),file=of)
print("reg inready;".format(),file=of)
print("reg [IN_DATA_LENGHT-1:0]indata;".format(),file=of)
print("wire outready;".format(),file=of)
print("wire earlyoutready;".format(),file=of)
print("wire [OUT_DATA_LENGHT-1:0]outdata;".format(),file=of)

In [5]:
print("////////////////instancing wrapping module///////////////////".format(),file=of)
print("{} {}(".format(GeneratedFileName,GeneratedFileName+"_inst"),file=of)
print(".clk(aclk),".format(),file=of)
print(".reset(!aresetn),".format(),file=of)
print(".enable(enable),".format(),file=of)
print(".inReady(inready),".format(),file=of)
for j in range (PR):
    for i in range(N):
        print(".AE{}R{}(indata[{}:{}]),".format( i, j, IN_DATA_LENGHT-((i+j*N)*(IN_WIDTH))-1 , IN_DATA_LENGHT-((i+j*N+1)*(IN_WIDTH)) ), file=of )
for i in range(N):
    print(".VE{}(indata[{}:{}]),".format( i,  (IN_DATA_LENGHT-(PR*N+i)*(IN_WIDTH))-1 , (IN_DATA_LENGHT-(PR*N+i+1)*(IN_WIDTH)) ), file=of )
print(".E{}toE{}Ready(outready),".format(0,PR-1),file=of)
for j in range (PR):
        print(".E{}(outdata[{}:{}]),".format( j, OUT_DATA_LENGHT-(j*OUT_WIDTH+1) ,  OUT_DATA_LENGHT-((j+1)*OUT_WIDTH) ), file=of)
print(".earlyE{}toE{}Ready(earlyoutready)".format(0,PR-1),file=of)
print(");".format(),file=of)

In [6]:
#in this cell we will add Axi stream compatability codes to wrapp the generated module in the previeous cell
print("/////////////////Main body/////////////".format(),file=of)
print("always @(posedge aclk)begin".format(),file=of)
print(" if(aresetn==0)begin".format(),file=of)
print("  m_axi_data<=0;".format(),file=of)
print("  m_axi_valid<=0;".format(),file=of)
print(" end".format(),file=of)
print(" else begin".format(),file=of)
print("  if(m_axi_ready==1 && m_axi_valid==1)begin".format(),file=of)
print("   m_axi_valid<=0;".format(),file=of)
print("  end".format(),file=of)
print("  else if(outready==1)begin".format(),file=of)
print("   m_axi_valid<=1;".format(),file=of)
print("   m_axi_data<=outdata;".format(),file=of)
print("  end".format(),file=of)
print(" end".format(),file=of)
print("end".format(),file=of)
print("always @(posedge aclk)begin".format(),file=of)
print(" if(aresetn==0)begin".format(),file=of)
print("  s_axi_ready<=1;".format(),file=of)
print("  inready<=0;".format(),file=of)
print("  indata<=0;".format(),file=of)
print(" end".format(),file=of)
print(" else begin".format(),file=of)
print("  inready<=0;".format(),file=of)
print("  if(s_axi_valid==1 && s_axi_ready==1)begin".format(),file=of)
print("   s_axi_ready<=0;".format(),file=of)
print("   inready<=1;".format(),file=of)
print("   indata<= s_axi_data;".format(),file=of)
print("  end".format(),file=of)
print("  else if(m_axi_valid==1 && m_axi_ready==1)begin".format(),file=of)
print("   s_axi_ready<=1;".format(),file=of)
print("  end".format(),file=of)
print(" end".format(),file=of)
print("end".format(),file=of)
print("\nendmodule",file=of)
of.close()